In [1]:
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

In [2]:
##Package was modified to allow complete cross mating
#library("devtools")
#install_github("gglinzijie/xbreed")
library("xbreed")

("|-----------------------------------------------------|")
("|                      xbreed                         |")
("|    Genomic simulation of purebreds and crossbreds   |")
("|               March 2017 Version 1.0.1              |")
("|                                                     |")
("|             H.Esfandyari,A.C.Sorensen               |")
("| Center for Quantitative Qenetics and Genomics (QGG) |")
("|             Aarhus University,Denmark               |")
("|                                                     |")
("|-----------------------------------------------------|")
("|Questions and bugs: esfandyari.hadi@gmail.com        |")
("|Development of xbreed was supported by GenSAP.       |")
("|-----------------------------------------------------|")


# Genome specification

In [6]:
#Number of markers per chr from chapter 1
lin_map<-read.table("raw.map")
m=1:22
for(i in 1:22) {m[i]=dim(lin_map[lin_map$V1==i,])[1]}
sum(m)

[1] 3928

In [4]:
#data from chaptre 1
geno<-read.table("Hetero_realigned_cov10_filtered3.raw")
pheno<-read.csv("2017heteroPheno.csv", header=T)
attach(pheno)

In [5]:
#parameter of genome
no.chr<-22
genome<-data.frame(matrix(NA, nrow=no.chr, ncol=6))
names(genome)<-c("chr","len","nmrk","mpos","nqtl","qpos")
genome$chr<-c(1:no.chr) #Chromosome id from 1 to 22
genome$len<-c(200,rep(100,21))#Chromosome length in cM
genome$nmrk<-c(3*m) #Number of markers, 3*3928 in total 
genome$mpos<-c('even') 
genome$nqtl<-c(40) #Number of qtl  40*22 = 880 in total
genome$qpos<-c('rnd')

# Historiacal population

In [6]:
#Historical population 
hp<-make_hp(hpsize=1000 ,ng=5000,h2=0.654,d2=0,phen_var=84,
            genome=genome,mutr=2.5*10**-4,laf=1)

---sel_seq_qtl is missing, it has been set to default value of 0
---sel_seq_mrk is missing, it has been set to default value of 0
Historical pop is initialized...
Simulating trait ...
Output data preparation ...
Establishment of historical population completed


In [7]:
#validation
mutr<-2.5*10**-4
ne<-1000
k<-2 
Fneu<-4*ne*mutr
(Expected_het1<-1-((1+((Fneu)/(k-1)))/(1+((Fneu*k)/(k-1)))))
(het_observed<-mean(2*(hp$freqMrk[,3]*hp$freqMrk[,4])))

[1] 0.3333333

[1] 0.398942

In [8]:
Male_founders<-data.frame(number=10,select='rnd')
Female_founders<-data.frame(number=10,select='rnd')

In [9]:
Selection<-data.frame(matrix(NA, nrow=2, ncol=3))
names(Selection)<-c('Number','type','Value') 
Selection$Number[1:2]<-c(10,10)
Selection$type[1:2]<-c('rnd','rnd')
Selection$Value[1:2]<-c('l','l')

In [10]:
sh_output<-data.frame(matrix(NA, nrow=1, ncol=4))
names(sh_output)<-c("data","qtl","freq_mrk","marker")
sh_output[1]<-c(1) 
sh_output[2]<-c(1) 
sh_output[3]<-c(1)
sh_output[4]<-c(1)

In [11]:
#10 sires and 10 dams perform complete cross; 200 eggs were produced per dam.
RP<-sample_hp(hp_out=hp,Male_founders= Male_founders,
              Female_founders=Female_founders,ng=1,Selection=Selection, 
              litter_size=10,saveAt="SNP3928",sh_output=sh_output,Display=FALSE)

Controlling input data ...
Intializing base population ...
Generation 0 started ......... 
Generation 0 is finished. Time taken: 14.86296
Generation 1 started ......... 
Generation 1 is finished. Time taken: 21.10412
Output data preparation ...
Writing output files ...
Sampling hp is done!


In [12]:
#function for calculate the allele coding, which is 0, 1, 2
bin_snp<-function(mat){
s1<-seq(1,ncol(mat),2)
s2<-seq(2,ncol(mat),2)
a1<-mat[,s1]+mat[,s2]
a1[a1==3]=1
a1[a1==4]=0
snp_code<-a1
return(snp_code)
 }

In [13]:
##geno and pheno from simulated data
pheno<-RP$output[[2]]$data$phen
tbvp<-RP$output[[2]]$data$tbvp
n<-bin_snp(RP$output[[2]]$mrk[,3:23570])
x<-as.matrix(n)-1
colnames(x)<-1:(3928*3)
row.names(x)<-1:1000

In [14]:
write.table(x,"2.4_geno.txt")
write.table(pheno,"2.4_pheno.txt")
write.table(tbvp,"2.4_tbvp.txt")

In [1]:
# Simulation result
x<-as.matrix(read.table("2.4_geno.txt"))
pheno<-as.numeric(unlist(read.table("2.4_pheno.txt")))
tbvp<-as.numeric(unlist(read.table("2.4_tbvp.txt")))

In [2]:
##load packages
library(doParallel)
library(foreach)
cl<-makeCluster(8) 
repeats <- 10
n.fold <- 5 
acc<-list()
library(rrBLUP)
packageVersion("rrBLUP") 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


[1] '4.6'

In [3]:
cal_se<-function(acc){sd(acc)/sqrt(repeats-1)}

In [4]:
# Function for calculating the accuracy of GP with 
# varying size of reference population and varying denstiy of SNP panel
cal_acc<-function(x,n.sample,n.snp){
set.seed(100)
id<-sample(1:dim(x)[1],n.sample)
snpId<-sample(1:dim(x)[2],n.snp)
#relationship matrix (Endelman at al. 2011)
A <- A.mat(x[id,snpId], n.core=8)
row.names(A)=1:n.sample;colnames(A)=1:n.sample
data <- data.frame(tbcw=pheno[id],tbvp=tbvp[id],gid=1:n.sample)
result<-data.frame(heritability=rep(NA,50),
                   bias_r=rep(NA,50),
                   unbias_r=rep(NA,50),
                   true_r=rep(NA,50))    
registerDoParallel(cl)
foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="rrBLUP") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         res <- kin.blup(bcw_test, K=A, geno="gid", pheno="tbcw")
         result[5*(j-1)+i,1]<-res$Vg/(res$Vg+res$Ve)
         result[5*(j-1)+i,2]<-cor(data$tbcw[id==i],res$pred[id==i])
         result[5*(j-1)+i,3]<-cor(data$tbcw[id==i],res$pred[id==i])/sqrt(res$Vg/(res$Vg+res$Ve))
         result[5*(j-1)+i,4]<-cor(data$tbvp[id==i],res$pred[id==i])
    }
   }
stopImplicitCluster()
return(c(apply(result, 2, mean),apply(result,2,cal_se)))    
}

In [7]:
sum_m<-sum(m)
n.snp<-c(round(0.1*sum_m),round(0.2*sum_m), 
             round(0.3*sum_m),round(0.4*sum_m),
             round(0.5*sum_m),round(0.6*sum_m),
             round(0.7*sum_m),round(0.8*sum_m),
             round(0.9*sum_m),sum_m,
             2*sum_m, 3*sum_m)
             
n.sample<-rep(c(500),12)
pair<-data.frame(n.snp=n.snp,n.sample=n.sample)

In [8]:
#implemente the caculation 
all_acc<-matrix(NA,nrow = 8,ncol = length(n.snp))
all_accBA<-all_accBB<-all_accBC<-all_accBL<-all_acc
for (i in 1:length(n.snp)){
    all_acc[,i]<-cal_acc(x,n.sample[i],n.snp[i])}

In [19]:
colnames(all_acc)<-n.snp
row.names(all_acc)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")

In [10]:
library("BGLR")
packageVersion("BGLR") 

[1] '1.0.5'

In [11]:
# Function for calculating the accuracy of GP with 
# varying size of reference population and varying denstiy of SNP panel
cal_accB<-function(x,n.sample,n.snp,M){
set.seed(100)
id<-sample(1:dim(x)[1],n.sample)
snpId<-sample(1:dim(x)[2],n.snp)
x1<-x[id,snpId]
data <- data.frame(tbcw=pheno[id],tbvp=tbvp[id],gid=1:n.sample)
result<-data.frame(heritability=rep(NA,50),
                   bias_r=rep(NA,50),
                   unbias_r=rep(NA,50),
                   true_r=rep(NA,50))                   
registerDoParallel(cl)
foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fm=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x1,model=M,saveEffects=T,saveAt='fm_')),nIter=2000,burnIn=1000,verbose=F)
         varU=var(x1%*%fm$ETA[[1]]$b)
         varE=fm$varE
         h2=varU/(varU+varE)
         result[5*(j-1)+i,1]<-h2
         result[5*(j-1)+i,2]<-cor(data$tbcw[id == i],fm$yHat[id == i])
         result[5*(j-1)+i,3]<-cor(data$tbcw[id==i],fm$yHat[id == i])/sqrt(h2)
         result[5*(j-1)+i,4]<-cor(data$tbvp[id==i],fm$yHat[id == i])
    }
   }
stopImplicitCluster()
return(c(apply(result, 2, mean),apply(result,2,cal_se)))    
}

In [12]:
for (i in 1:length(n.snp)){all_accBA[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BayesA")}

In [13]:
for (i in 1:length(n.snp)){all_accBB[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BayesB")}

In [14]:
for (i in 1:length(n.snp)){all_accBC[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BayesC")}

In [ ]:
for (i in 1:length(n.snp)){all_accBL[,i]<-cal_accB(x,n.sample[i],n.snp[i],"BL")}

In [16]:
colnames(all_accBA)<-n.snp
colnames(all_accBB)<-n.snp
colnames(all_accBC)<-n.snp
colnames(all_accBL)<-n.snp
row.names(all_accBA)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBB)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBC)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBL)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")

In [18]:
(sum<-list(GBLUP=all_acc,
                BA=all_accBA,
                BB=all_accBB,
                BC=all_accBC,
                BL=all_accBL))

,393,786,1178,1571,1964,2357,2750,3142,3535,3928,7856,11784
h2,0.363310541,0.44976490,0.50848833,0.49554165,0.51455202,0.52579918,0.53275068,0.51051618,0.51123960,0.52808219,0.55792586,0.54972511
bias_r,0.435327191,0.46824804,0.48504048,0.47648301,0.48073862,0.48390766,0.48682878,0.47732866,0.47688085,0.48109928,0.48787441,0.48279206
unbias_r,0.727108820,0.70122527,0.68301680,0.67983450,0.67279724,0.67003669,0.66994760,0.67141752,0.67022667,0.66517193,0.65685898,0.65535330
true_r,0.603680908,0.62532324,0.64376388,0.63969405,0.64593359,0.64612511,0.65105900,0.64418767,0.64377149,0.64709544,0.65177439,0.64906590
h2_se,0.009880405,0.01037233,0.01360362,0.01293085,0.01376099,0.01478906,0.01534238,0.01535742,0.01522410,0.01555636,0.01951229,0.02028223
bias_r_se,0.024242081,0.02230301,0.02213494,0.02257555,0.02193064,0.02207663,0.02229069,0.02316927,0.02318196,0.02275252,0.02242564,0.02258238
unbias_r_se,0.047232227,0.03831738,0.03522041,0.03658850,0.03433702,0.03430243,0.03494157,0.03715073,0.03690602,0.03575086,0.03487741,0.03578900
true_r_se,0.018344938,0.01885384,0.01762707,0.01836251,0.01783496,0.01797581,0.01765945,0.01821076,0.01853163,0.01785488,0.01727918,0.01730412
,393,786,1178,1571,1964,2357,2750,3142,3535,3928,7856,11784
h2,0.261422272,0.33691048,0.38656288,0.38563216,0.39029099,0.41026821,0.41485776,0.40542715,0.40338227,0.43397677,0.48590287,0.49015200


In [20]:
#save to xlsx file
library("xlsx")
write.xlsx(sum, "2_4_result.xlsx") 

In [ ]:
library("readxl")
plot<-read_excel("2_4_result.xlsx")